In [1]:
from torchvision import models, datasets, transforms as T
import torch
from PIL import Image
import numpy as np
from torchvision.models import ResNet50_Weights

In [3]:
resnet50 = models.resnet50(weights=ResNet50_Weights.DEFAULT)

In [4]:
# Pre-processing for ResNet-50 Inferencing, from https://pytorch.org/hub/pytorch_vision_resnet/
resnet50.eval()  
filename = 'buterfly.jpg' # change to your filename

input_image = Image.open(filename)
preprocess = T.Compose([
    T.Resize(256),
    T.CenterCrop(224),
    T.ToTensor(),
    T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
input_tensor = preprocess(input_image)
input_batch = input_tensor.unsqueeze(0) # create a mini-batch as expected by the model

In [11]:

# Export the model to ONNX
image_height = 224
image_width = 224
x = torch.randn(1, 3, image_height, image_width, requires_grad=True)
torch_out = resnet50(x)
torch.onnx.export(resnet50,                     # model being run
                  x,                            # model input (or a tuple for multiple inputs)
                  "resnet50.onnx",              # where to save the model (can be a file or file-like object)
                  export_params=True,           # store the trained parameter weights inside the model file
                  opset_version=12,             # the ONNX version to export the model to
                  do_constant_folding=True,     # whether to execute constant folding for optimization
                  input_names = ['input'],      # the model's input names
                  output_names = ['output'])    # the model's output names

In [5]:
# Inference with ONNX Runtime
import onnxruntime
from onnx import numpy_helper
import time

session_fp32 = onnxruntime.InferenceSession("resnet50.onnx", providers=['CPUExecutionProvider'])

In [6]:
# Read the categories
with open("imagenet_classes.txt", "r") as f:
    categories = [s.strip() for s in f.readlines()]

In [10]:
def softmax(x):
    """Compute softmax values for each sets of scores in x."""
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum()

latency = []
def run_sample(session, image_file, categories, inputs):
    start = time.time()
    input_arr = inputs.cpu().detach().numpy()
    ort_outputs = session.run([], {'input':input_arr})[0]
    latency.append(time.time() - start)
    output = ort_outputs.flatten()
    output = softmax(output) # this is optional
    top5_catid = np.argsort(-output)[:5]
    results= []
    for catid in top5_catid:
        print(categories[catid], output[catid])
        r = {
                "label": categories[catid],
                "probability": output[catid],
            }
        results.append(r)
    return results

In [11]:
ort_output = run_sample(session_fp32, 'buterfly.jpg', categories, input_batch)

ringlet 0.29918912
monarch 0.0960618
admiral 0.06821007
lycaenid 0.013846645
sulphur butterfly 0.008360029


In [12]:
ort_output

[{'label': 'ringlet', 'probability': 0.29918912},
 {'label': 'monarch', 'probability': 0.0960618},
 {'label': 'admiral', 'probability': 0.06821007},
 {'label': 'lycaenid', 'probability': 0.013846645},
 {'label': 'sulphur butterfly', 'probability': 0.008360029}]

In [9]:
filename = 'cow.jpg' # change to your filename

input_image = Image.open(filename)
preprocess = T.Compose([
    T.Resize(256),
    T.CenterCrop(224),
    T.ToTensor(),
    T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
input_tensor = preprocess(input_image)
input_batch = input_tensor.unsqueeze(0) # create a mini-batch as expected by the model
ort_output = run_sample(session_fp32, 'cow.jpg', categories, input_batch)

ox 0.40344307
bison 0.040098358
bighorn 0.016780484
water buffalo 0.01569149
hog 0.009256917


In [10]:
filename = 'beach.jpg' # change to your filename

input_image = Image.open(filename)
preprocess = T.Compose([
    T.Resize(256),
    T.CenterCrop(224),
    T.ToTensor(),
    T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
input_tensor = preprocess(input_image)
input_batch = input_tensor.unsqueeze(0) # create a mini-batch as expected by the model
ort_output = run_sample(session_fp32, 'beach.jpg', categories, input_batch)

seashore 0.16626585
sandbar 0.07883649
cliff 0.05631558
promontory 0.045474194
wreck 0.0038898492


In [11]:
filename = 'forest.jpg' # change to your filename

input_image = Image.open(filename)
preprocess = T.Compose([
    T.Resize(256),
    T.CenterCrop(224),
    T.ToTensor(),
    T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
input_tensor = preprocess(input_image)
input_batch = input_tensor.unsqueeze(0) # create a mini-batch as expected by the model
ort_output = run_sample(session_fp32, 'forest.jpg', categories, input_batch)

fountain 0.1369517
groom 0.09056474
castle 0.050991643
megalith 0.050879553
obelisk 0.029491441


In [12]:
filename = 'van.jpg' # change to your filename

input_image = Image.open(filename)
preprocess = T.Compose([
    T.Resize(256),
    T.CenterCrop(224),
    T.ToTensor(),
    T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
input_tensor = preprocess(input_image)
input_batch = input_tensor.unsqueeze(0) # create a mini-batch as expected by the model
ort_output = run_sample(session_fp32, 'van.jpg', categories, input_batch)

minibus 0.44485015
minivan 0.12285038
beach wagon 0.026604818
jeep 0.019635964
recreational vehicle 0.005178272


In [13]:
filename = 'road.jpg' # change to your filename

input_image = Image.open(filename)
preprocess = T.Compose([
    T.Resize(256),
    T.CenterCrop(224),
    T.ToTensor(),
    T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
input_tensor = preprocess(input_image)
input_batch = input_tensor.unsqueeze(0) # create a mini-batch as expected by the model
ort_output = run_sample(session_fp32, 'road.jpg', categories, input_batch)

bicycle-built-for-two 0.05923265
breakwater 0.057861704
unicycle 0.045096885
suspension bridge 0.043396775
streetcar 0.011755088


In [22]:
filename = '2persons.jpg' # change to your filename

input_image = Image.open(filename)
preprocess = T.Compose([
    T.Resize(256),
    T.CenterCrop(224),
    T.ToTensor(),
    T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
input_tensor = preprocess(input_image)
input_batch = input_tensor.unsqueeze(0) # create a mini-batch as expected by the model
ort_output = run_sample(session_fp32, '2persons.jpg', categories, input_batch)

notebook 0.4925105
laptop 0.21586348
suit 0.011487333
marimba 0.00931215
projector 0.006159393


In [23]:
filename = 'woman.jpg' # change to your filename

input_image = Image.open(filename)
preprocess = T.Compose([
    T.Resize(256),
    T.CenterCrop(224),
    T.ToTensor(),
    T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
input_tensor = preprocess(input_image)
input_batch = input_tensor.unsqueeze(0) # create a mini-batch as expected by the model
ort_output = run_sample(session_fp32, 'woman.jpg', categories, input_batch)

park bench 0.102493845
Labrador retriever 0.035666596
overskirt 0.015675696
Leonberg 0.011895797
cloak 0.01167709
